# Mapeo campos 

|Fuente | Sources |
|---|---|
|Nombre base para flora de Bogota | Taxa |
|Comentario de validación | Taxa.Comment |
|Para revisión | Taxa.CheckPriority 0 or 1 |
|Endemica | Taxa.Distribution |
|Estado de amenaza | RedListAssessments with Standard='JBB'
|Origen y status | Origins |
|Trabajo de No nativas Flora de Bogotá | Origins.Type???
|Determinador en fuente | Identifications.IdentifiedBy Merge with next field |
|Determinador Depurado | Identifications.IdentifiedBy |
|Calificador en fuente | Identifications.Certainty |
|Fecha_Determinacion en fuente | Identifications.Date |
|Fecha_Determinacion_(Year) | Identifications.Date |
|Fecha_Determinacion_(Month) | Identifications.Date |
|Fecha_Determinacion_(Day) | Identifications.Date |
|State | Areas |
|County | Areas |
|Localidad | Locations |
|Código de localidad georreferenciada | Locations.BD2020ID |
|Latitud_original | Locations.Latitude |
|Longitud_original | Locations.Longitud |
|Elev__minima | Locations.ElevationMin |
|Elev__maxima | Locations.ElevationMax |
|Latitud_decimal | Geocodings.InterpretedLat, Geocoder is DB_2020 |
|Longitud_decimal | Geocodings.InterpretedLon, Geocoder is DB_2020 |
|Latitud para trabajo de mapas | Geocodings.InterpretedLat, Geocoder is mapas_2020 |
|Longitud para trabajo de mapas | Geocodings.InterpretedLon, Geocoder is mapas_2020 |
|Extensión de georreferenciación | Geocodins.Uncertainty ???
|DATUM | Geocodings.Datum |
|Colector depurado | Occurrences.Collector |
|Colector principal | If no colector depurado, Occurrences.Collector, otherwise Occurrences.CollectorVerbatim  |
|Número de colector | Occurrences.CollectionNumber & Occurrences.CollectionNumberVerbatim |
|Fecha_Inicial_(Year) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(Month) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(day) | Occurrences.DateInit, Occurrences.DateEnd |
|Notas_Localidad_y_Habitat | Occurrences.Comment  |
|Fenologia | Occurrences.PhenoState |
|Descripcion ejemplar | Occurrences.Comment |
|Usos | Occurrences.Use |
|Nombres_comunes | Occurrences.CommonName |
|Comentarios generales | Occurrences.Comment |
|Referencia bibligráfica | Occurrence.Sources |
|Numero original en Base 2015 | Occurrence.DB2015ID |
|Origen para herbario | Specimens.Institution  |
|Número de identificación | Specimens.SpecimenCode |
|Fecha_Inicial | REMOVE |
|Notas de altura | REMOVE ???????
|Familia Revisada | REMOVE |
|Género revisado | REMOVE |
|Comentarios de determinación en fuente | REMOVE |
|Nivel taxonómico | REMOVE, info retrieved from TPRS |
|Familia en fuente | REMOVE  |
|Género en fuente | REMOVE  |
|Epíteto específico en fuente | REMOVE  |
|Autor del epíteto específico en fuente | REMOVE  |
|Epíteto infraespecífico | REMOVE  |
|Autor del infraespecífico en fuente | REMOVE  |
|Nombre científico en fuente | REMOVE  |
|Nombre validado | REMOVE  |
|Nivel taxonómico en fuente | REMOVE  |
|Herbario de proveniencia | REMOVE  |

In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

import requests
import json
import re
import time
from datetime import datetime

import pykew.powo as powo
from pygbif import species, registry

import credentials

In [267]:
conn_str = 'mysql+mysqlconnector://' + \
	f"{credentials.mysql['username']}:{credentials.mysql['password']}" + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [ ]:
#connection.execute(text('SET FOREIGN_KEY_CHECKS = 0'))

In [ ]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'
ori = pd.read_csv(orifile, dtype={'Elev__minima': str, 'Elev__maxima': str})

In [ ]:
tnrs_url = 'https://tnrsapi.xyz/tnrs_api.php'

In [ ]:
def tropicos(name, api_key):
	service = 'http://services.tropicos.org/name'
	r = requests.get(
		f'{service}/search/?apikey={api_key}&name={name}&format=json'
	)
	#s = requests.get(
	#	f'{service}/{r['NameId']}/highertaxa/?apikey={api_key}&name={name}&format=json'
	#)
	
	return r.json()


In [ ]:
def powo_query(name):
	act_name = ''
	bits = re.split(r'\s+', name)

	if bits[0] == '×' or bits[1] == '×':
		act_name = ' '.join(bits[:3])
	else:
		act_name = ' '.join(bits[:2])
	
	r = powo.search(act_name)

	if r.size() == 1:
		results = [i for i in r]
		return (results[0]['name'], results[0]['author'], results[0]['rank'])
	
	elif r.size() > 1:
		return f'Multiples results with name {name}'
	
	else:
		return f'No results with name {name}'

### Si la consulta a TNRS ya fue realizada y los resultados guardados:

In [ ]:
# Map between original name and corrected version by TNRS
taxa_map_file = 'tnrs_query.csv'
taxa_map = pd.read_csv(taxa_map_file)

### De lo contrario, aqui se realiza la consulta

In [ ]:
orinames = ori['Nombre base para flora de Bogota'].unique()
query = [[i, j] for i,j in enumerate(orinames) if pd.notna(j)]

r = requests.post(
	tnrs_url, 
	headers={"Content-Type": "application/json"},
	data=json.dumps({
		'data': query,
		'opts': {
			'sources': ['wcvp','wfo'],
			'class': 'wcvp',
			'mode': 'resolve',
			'matches' : 'best'
		}		
	})	
)

In [ ]:
taxa_map = pd.DataFrame.from_records(results)
taxa_map.to_csv('tnrs_query.csv', index=False)

In [ ]:
taxa_map.loc[taxa_map.Overall_score == '', 'Overall_score'] = 'NAN'
taxa_map['Overall_score'] = taxa_map.Overall_score.astype(np.float32)

# Aqui se continua la normalizacion de nombres

### Consulta a POWO de hibridos

Primero remover hibridos no publicados, solo se registran a nivel de determinacion

In [ ]:
taxa_map = taxa_map.drop(
	index=taxa_map.loc[taxa_map.Name_submitted.str.contains(r'Salvia [\w\.\s]+ ×')].index
)

In [ ]:
for row in taxa_map.loc[taxa_map.Name_submitted.str.contains('×')].itertuples():
	#print(row.Name_submitted)
	po = powo_query(row.Name_submitted)
	if isinstance(po, tuple):
		taxa_map.loc[row.Index, 'Name_matched'] = po[0]
		taxa_map.loc[row.Index, 'Author_matched'] = po[1]
		taxa_map.loc[row.Index, 'Name_matched_rank'] = po[2].lower()
	else:
		print(po)
	time.sleep(1)

In [ ]:
# Manually correct names couldn't be matched easily by POWO,
# but there is a simple solution there 

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Pelargonium × hortorum')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Pelargonium × hortorum', 'L.H.Bailey', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Abelia × grandiflora')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Abelia × grandiflora', '(Rovelli ex André) Rehder', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Lavatera × clementii')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Lavatera × clementii', 'Cheek', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Polygala × dalmasiana')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Polygala', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Epidendrum chicoense')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Epidendrum', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Plantae')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = (np.nan, np.nan, np.nan)

# TNRS couldn't get appropriate response for the following

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Callianthe striata')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Callianthe striata', '(G.F.Dicks. ex Lindl.) Donnell', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Phlegmariurus acifolius')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Phlegmariurus acifolius', '(Rolleri) B.Øllg.', 'species')


In [ ]:
# Save eliminated binomials from the database, they probably are invalid or unpublished names
taxa_map.loc[(taxa_map.Overall_score < 0.9) & (taxa_map.Name_matched_rank == 'genus'),
	['Name_submitted', 'Name_matched', 'Name_matched_rank']
	].to_csv('binomios_eliminados.csv', index=False)

In [ ]:
# Remove lichens: Ramalina, Usnea, Sticta, Candelariella
# Remove general indets: 'Pteridophyta', 'Magnoliophyta', 'Plantae', 'Tracheophyta'

taxa_map = taxa_map.drop(
	index=taxa_map[taxa_map.Name_submitted.isin([
		'Ramalina', 'Usnea', 'Sticta', 'Candelariella',
		'Pteridophyta', 'Magnoliophyta', 'Plantae', 'Tracheophyta'
	])].index
)

In [ ]:
taxa_map.to_csv('taxa_map.csv', index=False)

### Obtener jerarquía taxonómica de World Flora Online

In [ ]:
taxa_map = pd.read_csv('taxa_map.csv')

In [ ]:
wfo_file = '../../Botanical_databases/world_flora_online/classification.csv'

In [ ]:
datyp = {'localID':str, 'subfamily': str, 'tribe': str, 'subtribe': str, 
 	'subgenus':str, 'infraspecificEpithet': str, 'tplID': str}

In [ ]:
wfo = pd.read_csv(wfo_file, sep='\t', encoding='latin', dtype=datyp)

In [ ]:
taxa_in = taxa_map.groupby(['Name_matched','Author_matched']
	).size(
	).reset_index(
	).drop(columns=0
	).rename(columns={'Name_matched': 'Name', 'Author_matched':'Author'})

In [ ]:
# For some reason Passiflora tripartita is retrieved twice, one with the wrong authorship
taxa_in = taxa_in.drop(
	index=taxa_in[
		(taxa_in.Name == 'Passiflora tripartita') & 
		(taxa_in.Author == '(Juss.) Poir.')
	].index).reset_index()

In [ ]:
def get_family(name):
	ret = np.nan
	if pd.isna(name): return np.nan
	bits = re.split(r'\s+', name)
	nname = bits[0]
	out = wfo.loc[(wfo.scientificName == nname) & (wfo.taxonRank == 'genus')]

	if out.shape[0] == 1: 
		ret =  out['family'].item()
	
	elif out.shape[0] > 1:

		if out[out['nomenclaturalStatus'] == 'Valid'].shape[0] == 1:
			ret =  out.loc[out['nomenclaturalStatus'] == 'Valid', 'family'].item()

		elif out[out['taxonomicStatus'] == 'Accepted'].shape[0] == 1:
			ret =  out.loc[out['taxonomicStatus'] == 'Accepted', 'family'].item()

		else:
			if len(bits) > 1: nname = f'{bits[0]} {bits[1]}'
			out = wfo.loc[(wfo.scientificName == nname)]
			fams = out.family.unique()
			fams = [x for x in fams if pd.notna(x)]

			if len(fams) == 1:
				ret =  fams[0]		
		
	elif out.shape[0] < 1 and re.search(r'\w+aceae$', nname):
		out = wfo.loc[(wfo.scientificName == nname) & (wfo.taxonRank == 'family')]
		fams = out.family.unique()
		fams = [x for x in fams if pd.notna(x)]

		if len(fams) == 1:
			ret =  fams[0]

	return ret


In [ ]:
taxa_in['wfo_family'] = taxa_in.Name.apply(get_family)

In [ ]:
# Some missing names in World Flora Online
taxa_in.loc[taxa_in.Name == 'Hydrophyllaceae', 'wfo_family'] = 'Hydrophyllaceae'
taxa_in.loc[taxa_in.Name == 'Stemmatella', 'wfo_family'] = 'Asteraceae'
taxa_in.loc[taxa_in.Name == 'Coniza', 'wfo_family'] = 'Asteraceae'
taxa_in.loc[taxa_in.Name == '× Agropogon lutosus', 'wfo_family'] = 'Poaceae'


In [ ]:
famis = taxa_in.groupby('wfo_family'
	).size(
	).reset_index(
	)

In [ ]:
def get_papa(name):
	ret = np.nan

	if pd.notna(name):
		out = wfo.loc[(wfo.scientificName == name), 'parentNameUsageID']
		th = [x for x in out if pd.notna(x)]
		
		if len(th) == 1:
			parid = th[0]
			ret = wfo.loc[(wfo.taxonID == parid), 'scientificName'].item()
	
	return ret

In [ ]:
famis['wfo_order'] = famis.wfo_family.apply(get_papa)

In [ ]:
famis.loc[famis.wfo_family == 'Hydrophyllaceae', 'wfo_order'] = 'Solanales'

In [ ]:
famis['wfo_clade'] = famis.wfo_order.apply(get_papa)

In [ ]:
taxa_in = taxa_in.merge(famis, 'left', 'wfo_family')

In [ ]:
taxa_in.drop(columns=0).to_csv('taxa_in.csv', index=False)

### Verificación de nombres realizada y previamente guardada localmente

Abajo se formalizan las jerarquías dado el diseño de la tabla Taxa en la DB

In [ ]:
taxa_in = pd.read_csv('taxa_in.csv')

In [ ]:
taxa_in = taxa_in.reset_index(drop=True)

In [ ]:
taxa_in['TaxonID'] = taxa_in.index + 1

In [ ]:
taxa_in[['CheckPriority', 'Distribution', 'Parent']] = np.nan, np.nan, np.nan

In [ ]:
oriname = 'Nombre base para flora de Bogota'

for row in taxa_in.itertuples():
	
	qu = (ori[oriname] == row.Name) & (ori.Endemica == 'SI')
	
	if ori.loc[qu].shape[0] > 0:
		taxa_in.loc[taxa_in.Name == row.Name, 'Distribution'] = 0

	qu = (ori[oriname] == row.Name) & (ori['Para revisión'] == 'SI')
	
	if ori.loc[qu].shape[0] > 0:
		taxa_in.loc[taxa_in.Name == row.Name, 'CheckPriority'] = 5

In [ ]:
last_id = taxa_in.TaxonID.iloc[-1]

In [ ]:
for th in taxa_in.wfo_clade.unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['TaxonID'] = last_id
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec])
		])


In [ ]:
for th in taxa_in.wfo_order[taxa_in.wfo_order.notna()].unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		papa = taxa_in.loc[taxa_in.wfo_order == th, 'wfo_clade'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['Parent'] = papaid
		rec['TaxonID'] = last_id
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec])
		])


In [ ]:
for th in taxa_in.wfo_family[taxa_in.wfo_family.notna()].unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		papa = taxa_in.loc[taxa_in.wfo_family == th, 'wfo_order'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['Parent'] = papaid
		rec['TaxonID'] = last_id
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec])
		])
	else:
		papa = taxa_in.loc[taxa_in.wfo_family == th, 'wfo_order'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
		taxa_in.loc[taxa_in.Name == th, 'Parent'] = papaid



In [ ]:
no_gens = taxa_in.wfo_family.unique().tolist() + \
	taxa_in.wfo_order.unique().tolist() + \
	taxa_in.wfo_clade.unique().tolist() + ['×']

no_gens = [i for i in no_gens if pd.notna(i)]

In [ ]:
gens = [x for x in taxa_in.Name.apply(lambda x: re.split(r'\s+', x)[0]).unique()]
gens = [x for x in gens if not x in no_gens]

In [ ]:
for gn in gens:
	#print(gn)
	
	if pd.notna(gn):

		qu = taxa_in.loc[taxa_in.Name == gn]

		if qu.shape[0] == 0:
			last_id += 1
			papa = taxa_in.loc[taxa_in.Name.str.startswith(f'{gn} '), 'wfo_family'].iloc[0]
			#print(papa)
			papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
			#print(papaid)
			rec = {i:np.nan for i in taxa_in.columns}
			rec['Name'] = gn
			rec['Parent'] = papaid
			rec['TaxonID'] = last_id
			taxa_in = pd.concat([
				taxa_in, 
				pd.DataFrame.from_records([rec])
			])

		else:
			papa = taxa_in.loc[taxa_in.Name == gn, 'wfo_family'].iloc[0]
			#print(papa)
			papaid = taxa_in.loc[taxa_in.Name == papa, 'TaxonID'].iloc[0]
			#print(papaid)
			taxa_in.loc[taxa_in.Name == gn, 'Parent'] = papaid



Por alguna razón Vriesea está incorrectamente clasificada

In [ ]:
taxa_in.loc[taxa_in.Name == 'Bromeliaceae']

In [ ]:
taxa_in.loc[taxa_in.Name == 'Vriesea', 'Parent'] = 3230

In [ ]:
for sp in taxa_in.loc[
	taxa_in.Name.str.contains(r'^[\w\-]+\s+\w+') |
	taxa_in.Name.str.contains(r'\w+\s+×'),
	'Name'
	].unique():

	#print(sp)
	gn = re.split(r'\s+', sp)[0]
	#print(gn)
	qu = taxa_in.loc[taxa_in.Name == gn]
	papaid = taxa_in.loc[taxa_in.Name == gn, 'TaxonID'].iloc[0]
	#print(papaid)
	taxa_in.loc[taxa_in.Name == sp, 'Parent'] = papaid

In [ ]:
taxa_in.loc[taxa_in.Name == '× Agropogon lutosus', 'Parent'
	] = taxa_in.loc[taxa_in.Name == 'Andropogon'].index[0]

In [ ]:
taxa_in[['AcceptedName', 'Authority', 'Protologue', 'Comment']
	] = np.nan, np.nan, np.nan, np.nan

In [ ]:
taxa_in[['TaxonID', 'Name', 'Author', 'CheckPriority', 'Distribution', 'Parent',
	'AcceptedName', 'Authority', 'Protologue', 'Comment']].to_csv(
	'taxa_in.csv', index=False)

# Insert Taxa table

If all nomenclatural and classification validation has been conducted, read `taxa_in.csv` file and directly do insertion.

In [2]:
taxa_in = pd.read_csv('taxa_in.csv')

In [11]:
taxa_in[['TaxonID', 'Name', 'Author', 'CheckPriority', 'Distribution', 'Parent',
	'AcceptedName', 'Authority', 'Protologue', 'Comment']
	].to_sql('Taxa', engine, if_exists='append', index=False, chunksize=10000, method='multi')

-1

# Procedamiento tablas Locations y Geocodings

| Campo original | Campo DB |
|---|---|
|Localidad | Locations |
|Código de localidad georreferenciada | Locations.BD2020ID |
|Latitud_original | Locations.Latitude |
|Longitud_original | Locations.Longitud |
|Elev__minima | Locations.ElevationMin |
|Elev__maxima | Locations.ElevationMax |
|Latitud_decimal | Geocodings.InterpretedLat, Geocoder is DB_2020 |
|Longitud_decimal | Geocodings.InterpretedLon, Geocoder is DB_2020 |
|Latitud para trabajo de mapas | Geocodings.InterpretedLat, Geocoder is mapas_2020 |
|Longitud para trabajo de mapas | Geocodings.InterpretedLon, Geocoder is mapas_2020 |
|Extensión de georreferenciación | Geocodins.Uncertainty ???
|DATUM | Geocodings.Datum |


In [268]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'

colpp = {
	'Colector depurado': 'colde', 'Colector principal': 'colpri', 
	'Número de colector': 'CollectionNumberVerbatim', 
	'Fecha_Inicial_(Year)': 'year',
	'Fecha_Inicial_(Month)': 'month', 'Fecha_Inicial_(day)': 'day',
	'Notas_Localidad_y_Habitat': 'notas', 'Fenologia': 'feno',
	'Usos': 'usos', 'Nombres_comunes': 'comname', 
	'Comentarios generales': 'comm', 'Referencia bibligráfica': 'ref',
	'Numero original en Base 2015': 'dbid', 
	'Nombre base para flora de Bogota': 'name',
	'Descripcion ejemplar': 'descr', 'Localidad': 'Localidad',
	'Latitud_decimal': 'Latitud_decimal', 'Longitud_decimal': 'Longitud_decimal',
	'Código de localidad georreferenciada': 'DB2020ID',
	'Latitud_original': 'Latitude_text', 'Longitud_original': 'Longitude_text',
	'Elev__minima': 'ElevationMin', 'Elev__maxima': 'ElevationMax',
	'State': 'State', 'County': 'County',
	}

ori = pd.read_csv(orifile, dtype={'Elev__minima': str, 'Elev__maxima': str},
	usecols=colpp)

ori = ori.rename(columns=colpp)

In [269]:
ori['Localidad'] = ori.Localidad.str.replace(r'\s+', ' ', regex=True
	).str.replace(r'(^\s+|\s+$)', '', regex=True
	)

ori['Latitude_text'] = ori.Latitude_text.str.replace(',', '.')
ori['Longitude_text'] = ori.Longitude_text.str.replace(',', '.')

bits = ori.Latitude_text.str.extractall(r'([\d\.]+)'
	).reset_index(
	).pivot(index='level_0', columns='match', values=0
	)

bits = bits.fillna('0') 

bits['Latitude'] = (bits[0].astype(np.float64) + 
		bits[1].astype(np.float64) / 60 + 
		bits[2].astype(np.float64) / 3600) 

ori = ori.merge(bits[['Latitude']], how='left', left_index=True, right_index=True
	)

bits = ori.Longitude_text.str.extractall(r'([\d\.]+)'
	).reset_index(
	).pivot(index='level_0', columns='match', values=0
	)

bits = bits.fillna('0') 

bits['Longitude'] = -1 * (bits[0].astype(np.float64) + 
		bits[1].astype(np.float64) / 60 + 
		bits[2].astype(np.float64) / 3600) 

ori = ori.merge(bits[['Longitude']], how='left', left_index=True, right_index=True)
ori['County'] = ori.County.fillna('')
ori['State'] = ori.State.fillna('')
ori['Localidad'] = ori.Localidad.fillna('')
ori['Localidad'] = ori.State + ', ' + ori.County + ', ' + ori.Localidad 

ori['Localidad'] = ori.Localidad.str.replace(r'\s+', ' ', regex=True
	).str.replace(r'(, )+', ', ', regex=True
	).str.replace(r'(^[\.,\s]+|[\.,\s]+$)', '', regex=True
	).str.replace('Bogota', 'Bogotá', regex=False
	)

ori.loc[
	~ori.State.str.contains(r'(bogot|distrit)', flags=re.I, na=False) &
	~ori.County.str.contains(r'(bogot|distrit|Usaqu|Usme)', flags=re.I, na=False),
	'State'
	] = 'Cundinamarca'

ori.loc[
	ori.State.str.contains(r'(bogot|distrit)', flags=re.I, na=False) |
	ori.County.str.contains(r'(bogot|distrit|Usaqu|Usme)', flags=re.I, na=False),
	['State', 'County'] 
	] = ['Bogotá', 'Bogotá']

ori.loc[ori.County == '', 'County'] = np.nan
ori['ElevationMax'] = ori.ElevationMax.str.replace(r'\s', '', regex=True)
ori.loc[ori.ElevationMax == '', 'ElevationMax'] = np.nan
ori['ElevationMax'] = ori.ElevationMax.astype(np.float64)
ori['ElevationMin'] = ori.ElevationMin.astype(np.float64)

/tmp/ipykernel_305037/3942084273.py:46: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~ori.State.str.contains(r'(bogot|distrit)', flags=re.I, na=False) &
/tmp/ipykernel_305037/3942084273.py:47: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~ori.County.str.contains(r'(bogot|distrit|Usaqu|Usme)', flags=re.I, na=False),
/tmp/ipykernel_305037/3942084273.py:52: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ori.State.str.contains(r'(bogot|distrit)', flags=re.I, na=False) |
/tmp/ipykernel_305037/3942084273.py:53: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ori.County.str.contains(r'(bogot|distrit|Usaqu|Usme)', flags=re.I, na=False),


In [270]:
locs = ori.groupby(
	['DB2020ID', 'Localidad', 'Latitude', 'Longitude', 'ElevationMin', 'ElevationMax'],
	dropna=False
	).size(
	).reset_index(
	).drop(columns=0
	)

locs['LocationID'] = locs.index + 1

ori = ori.merge(
	locs, 
	how='left', 
	on=[
		'DB2020ID', 'Localidad', 'Latitude', 'Longitude', 'ElevationMin', 
		'ElevationMax'
	])

In [271]:
locs.rename(
		columns={'Localidad':'Name'}
	).to_sql(
		'Locations', engine, if_exists='append', index=False, chunksize=10000, 
		method='multi'
	)

-2

# Procesamiento tabla Sources

In [151]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'

colpp = {
	'Colector depurado': 'colde', 'Colector principal': 'colpri', 
	'Número de colector': 'CollectionNumberVerbatim', 
	'Fecha_Inicial_(Year)': 'year',
	'Fecha_Inicial_(Month)': 'month', 'Fecha_Inicial_(day)': 'day',
	'Notas_Localidad_y_Habitat': 'notas', 'Fenologia': 'feno',
	'Usos': 'usos', 'Nombres_comunes': 'comname', 
	'Comentarios generales': 'comm', 'Referencia bibligráfica': 'ref',
	'Numero original en Base 2015': 'dbid', 
	'Nombre base para flora de Bogota': 'name',
	'Descripcion ejemplar': 'descr'
	}

ori = pd.read_csv(orifile, dtype={'Elev__minima': str, 'Elev__maxima': str},
	usecols=colpp)

ori = ori.rename(columns=colpp)

In [152]:
ori['author'] = None

ori.loc[ori.ref.notna(), 'author'] = ori.loc[ori.ref.notna(), 'ref'
	].str.extract(r'^([^\(]+)\(').loc[:, 0]

authors = ori.groupby('author').size().reset_index().drop(columns=0)
authors.index += 1
authors['authorID'] = authors.index

tmp = authors.author.str.extractall(r'([\w\-\.]{3,},( [A-Z]\.)+)'
	).drop(columns=1).rename(columns={0:'person'})

persons = tmp.groupby('person').size().reset_index().drop(columns=0)
persons['PersonID'] = persons.index + 1 
persons['LastName'] = persons.person.str.extract(r'(^[\w\-\.]+),')
persons['FirstName'] = persons.person.str.extract(r',(.+)$')
persons = persons.rename(columns={'person':'NameVerbatim'})

peoper = tmp.reset_index().merge(
	persons, left_on='person', right_on='NameVerbatim', how='left'
	).rename(columns={
		'level_0': 'People', 'match':'Order', 'PersonID':'Person'
	})

peoper['Order'] += 1
sour = ori.groupby(['ref', 'author']).size().reset_index().drop(columns=0)
sour = sour.merge(authors, how='left', on='author')
sour['Year'] = sour.ref.str.extract(r'\((\d+)\)')
sour['Name'] = sour.ref.str.extract(r'\).\s+(.+)[\.\s]*$')
sour['Type'] = 'Document'
# Removing a ghost reference (probably a specimen in F)
sour = sour.drop(
	index=sour[sour.Name.isna()].index
	)
sour = sour.reset_index(drop=True)
sour['SourceID'] = sour.index + 1
ori = ori.merge(sour, on='ref', how='left')

In [124]:
persons.to_sql('Persons', engine, if_exists='append', index=False, chunksize=10000, method='multi')

authors.rename(columns={'authorID':'PeopleID'})[['PeopleID']
	].to_sql('People', engine, if_exists='append', index=False, chunksize=10000, 
		method='multi'
	)

peoper[['People', 'Order', 'Person']].to_sql(
	'PeoplePersons', engine, if_exists='append', index=False, chunksize=10000, 
		method='multi')

sour.rename(columns={'authorID':'Author'})[['SourceID', 'Type', 'Author', 'Year', 'Name']
	].to_sql('Sources', engine, if_exists='append', index=False, chunksize=10000, method='multi')

-1

# Procesamiento tabla ocurrencias

| Campo | Nota |
|---|---|
|Colector depurado | Occurrences.Collector |
|Colector principal | If no colector depurado, Occurrences.Collector, otherwise Occurrences.CollectorVerbatim  |
|Número de colector | Occurrences.CollectionNumber & Occurrences.CollectionNumberVerbatim |
|Fecha_Inicial_(Year) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(Month) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(day) | Occurrences.DateInit, Occurrences.DateEnd |
|Notas_Localidad_y_Habitat | Occurrences.Comment  |
|Fenologia | Occurrences.PhenoState |
|Descripcion ejemplar | Occurrences.Comment |
|Usos | Occurrences.Use |
|Nombres_comunes | Occurrences.CommonName |
|Comentarios generales | Occurrences.Comment |
|Referencia bibligráfica | Occurrence.Sources |
|Numero original en Base 2015 | Occurrence.DB2015ID |

In [153]:
ori.loc[ori.colde.isna(), 'colde'] = ori.loc[ori.colde.isna(), 'colpri']

In [154]:
bits = ori.loc[ori.CollectionNumberVerbatim.notna()
	& ori.CollectionNumberVerbatim.str.contains(r'[^\d]'), 'CollectionNumberVerbatim'
	].str.extractall(r'(\d+)'
	).reset_index(
	).pivot(index='level_0', columns='match', values=0
	)

ori = ori.merge(bits, 'left', left_index=True, right_index=True
	)

ori['CollectionNumber'] = np.nan

# Sorting number before spliting token
ori.loc[ori.CollectionNumberVerbatim.notna() & ori[1].notna()
	& ori.CollectionNumberVerbatim.str.contains(r'\d+[^\d]+\d{1,2}([^\d]+$|$)'),
	'CollectionNumber'] = \
	ori.loc[ori.CollectionNumberVerbatim.notna() & ori[1].notna()
		& ori.CollectionNumberVerbatim.str.contains(r'\d+[^\d]+\d{1,2}([^\d]+$|$)'), 
		0].astype(np.int64)

# Sorting number after spliting token
ori.loc[ori.CollectionNumberVerbatim.notna() & ori[1].notna()
	& ori.CollectionNumberVerbatim.str.contains(r'[^\d]\d{3,}'),
	'CollectionNumber'] = \
	ori.loc[ori.CollectionNumberVerbatim.notna() & ori[1].notna()
		& ori.CollectionNumberVerbatim.str.contains(r'[^\d]\d{3,}'), 
		1].astype(np.int64)

# Remove non-digits from collections numbers
ori.loc[ori.CollectionNumber.isna() & 
	ori.CollectionNumberVerbatim.str.contains(r'(^[^\d]+\d+$|^\d+[^\d]+$)'), 
	'CollectionNumber'
	] = ori.loc[ori.CollectionNumber.isna() & 
	ori.CollectionNumberVerbatim.str.contains(r'(^[^\d]+\d+$|^\d+[^\d]+$)'), 
	'CollectionNumberVerbatim'
	].str.replace(r'[^\d]+', '', regex=True).astype(np.int64)

# Remove non-digits from collections numbers (again)
ori.loc[ori.CollectionNumber.isna() & 
	ori.CollectionNumberVerbatim.str.contains(r'[^\d]'), 
	'CollectionNumber'] = ori.loc[ori.CollectionNumber.isna() & 
	ori.CollectionNumberVerbatim.str.contains(r'[^\d]'), 
	'CollectionNumberVerbatim'].str.replace(r'[^\d]', '', regex=True
	).astype(np.int64)

# Sorting number is verbatim number
ori.loc[ori.CollectionNumber.isna() & ori.CollectionNumberVerbatim.notna(), 'CollectionNumber'] = ori.loc[ori.CollectionNumber.isna() & ori.CollectionNumberVerbatim.notna(), 'CollectionNumberVerbatim'].astype(np.int64)

/tmp/ipykernel_27363/1556294952.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  & ori.CollectionNumberVerbatim.str.contains(r'\d+[^\d]+\d{1,2}([^\d]+$|$)'),
/tmp/ipykernel_27363/1556294952.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  & ori.CollectionNumberVerbatim.str.contains(r'\d+[^\d]+\d{1,2}([^\d]+$|$)'),
/tmp/ipykernel_27363/1556294952.py:34: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ori.CollectionNumberVerbatim.str.contains(r'(^[^\d]+\d+$|^\d+[^\d]+$)'),
/tmp/ipykernel_27363/1556294952.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ori.CollectionNumberVerbatim.str.contains(r'(^[^\d]+\d+$|^\d+[^\d]+$)'),


In [155]:
# Number of days per month
ld = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
# Date

def get_date(row):
	out = [None, None]

	if pd.notna(row.year) and row.year >= 1:

		if pd.notna(row.month) and row.month >= 1:
		
			if pd.notna(row.day) and row.day >= 1:

				if row.year % 2 > 0 and row.month == 2 and row.day > 28:
					out = [
						datetime.strptime(f"{int(row.year)}-{int(row.month)}-28", "%Y-%m-%d"),
						datetime.strptime(f"{int(row.year)}-{int(row.month)}-28", "%Y-%m-%d")
					]
				else:
					out = [
						datetime.strptime(f"{int(row.year)}-{int(row.month)}-{int(row.day)}", "%Y-%m-%d"),
						datetime.strptime(f"{int(row.year)}-{int(row.month)}-{int(row.day)}", "%Y-%m-%d")
					]
					
			else:
				out = [
					datetime.strptime(f"{int(row.year)}-{int(row.month)}-01", "%Y-%m-%d"),
					datetime.strptime(f"{int(row.year)}-{int(row.month)}-{ld[int(row.month)]}", "%Y-%m-%d")
				]
		
		else:
			out = [
					datetime.strptime(f"{int(row.year)}-01-01", "%Y-%m-%d"),
					datetime.strptime(f"{int(row.year)}-12-31", "%Y-%m-%d")
				]
	
	return out

ori[['DateInit', 'DateEnd']] = ori.apply(get_date, axis=1, result_type='expand')

In [156]:
repmap = {'fértil': np.nan, 'Esteril': 'estéril', 'Flor': 'flor', 
	'Flor & Fruto': 'flor y fruto', 
 	'Estéril': 'estéril', 'Fruto': 'fruto', 'Fertil': 'flor y fruto', 
	'Fértil': 'flor y fruto', 'Infertil': 'estéril', 'Botón floral': 'flor', 
	'Flor/Fruto': 'flor y fruto'
}

ori['PhenoState'] = ori.feno.replace(repmap)

In [157]:
ori['comname'] = ori.comname.str.replace('"', ''
	).str.replace(r'\s+', ' ', regex=True
	).str.replace(r'^\s+', '', regex=True
	).str.replace(r'\s+$', '', regex=True
	).str.lower(
	)

In [158]:
# A couple of descriptions inserted in the wrong column
ori.loc[ori.usos.notna() & ori.usos.str.contains('Hierba'), 'descr'] = \
ori.loc[ori.usos.notna() & ori.usos.str.contains('Hierba'), 'usos']

ori.loc[ori.usos.notna() & ori.usos.str.contains('Hierba'), 'usos'] = np.nan

In [159]:
ori['comm'] = ori.comm.str.replace(r'[‘”“"\r\n]', '', regex=True
	).str.replace(r'\s{2,}', ' ', regex=True
	)

In [160]:
ori['notas'] = ori.notas.str.replace(r'[‘”“"\r\n]', '', regex=True
	).str.replace(r'\s{2,}', ' ', regex=True
	)

In [161]:
# Remove extrmely long poetic plant descriptions longer than 500 chars
ori.loc[ori.descr.notna() & ori.descr.apply(lambda x: len(x) > 500 if pd.notna(x) else 0), 'descr'] = np.nan

In [162]:
ori['Comment'] = ori.descr.apply(str) + "|" + \
	ori.notas.apply(str) + "|" + \
	ori.comm.apply(str) + "|"

In [163]:
ori.columns

Index([                    'name',                   'colpri',
                          'colde', 'CollectionNumberVerbatim',
                           'year',                    'month',
                            'day',                    'notas',
                           'feno',                    'descr',
                           'usos',                  'comname',
                           'comm',                      'ref',
                           'dbid',                 'author_x',
                       'author_y',                 'authorID',
                           'Year',                     'Name',
                           'Type',                 'SourceID',
                                0,                          1,
                                2,         'CollectionNumber',
                       'DateInit',                  'DateEnd',
                     'PhenoState',                  'Comment'],
      dtype='object')

In [167]:
ori[['colde', 'CollectionNumberVerbatim', 'CollectionNumber', 'DateInit', 'DateEnd', 'PhenoState',
	 'comname', 'Comment', 'SourceID', 'usos', 'dbid']].head()

#Reference

,colde,CollectionNumberVerbatim,CollectionNumber,DateInit,DateEnd,PhenoState,comname,Comment,SourceID,usos,dbid
0,NaN,NaN,NaN,1999-02-10,1999-02-10,NaN,NaN,"Hierba de 1 metro de altura, inflorescencias t...",NaN,NaN,NaN
1,NaN,NaN,NaN,1999-02-18,1999-02-18,NaN,NaN,Hierba rastrera. Botón floral amarillo|En suel...,NaN,NaN,NaN
2,NaN,NaN,NaN,NaT,NaT,NaN,NaN,Liana con hojas discóloras con indumento blanc...,NaN,NaN,NaN
3,NaN,NaN,NaN,1999-02-10,1999-02-10,NaN,NaN,Arbusto de 2 metros de altura. Espinas en hoja...,NaN,NaN,NaN
4,NaN,NaN,NaN,NaT,NaT,NaN,NaN,"Hierba erecta 1 m. Fronda 1.5 , larga|Parcela ...",NaN,NaN,NaN


________

In [ ]:
# Multispecies hybrids

taxa_map.loc[taxa_map.Name_submitted.str.contains('×') & 
	taxa_map.Name_matched.notna() & 
	~taxa_map.Name_matched.str.contains(r'×',na=False)]

-------

In [266]:
df = pd.read_sql(sql='SELECT * FROM Taxa', con=connection)

In [ ]:
df

In [ ]:
#connection.execute(text('SET FOREIGN_KEY_CHECKS = 1'))

In [249]:
connection.close()